In [66]:
import os
import json
import sys 
from sys import exit 
import numpy as np

In [67]:
import time
import urllib.request
import ssl
from dotenv import load_dotenv
from unicodedata import name
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from urllib.parse import urlparse, parse_qs

In [68]:
load_dotenv()

True

In [69]:
twitter_user = os.getenv("twitter_user")
twitter_password = os.getenv("twitter_password")

service = Service('E:/Users/1167486/Local/Drivers_web/chromedriver-win64/chromedriver')
options = webdriver.ChromeOptions()
options.binary_location = r'E:/Users/1167486/Local/Drivers_web/chrome-win64/chrome.exe'

driver = webdriver.Chrome(service=service, options=options)


In [70]:
driver.maximize_window()
driver.get('https://x.com/i/flow/login')
time.sleep(10)

In [72]:
username = driver.find_element("css selector", "input[autocomplete='username']")


In [73]:
username.clear()


In [74]:
username.send_keys(twitter_user)
username.send_keys(Keys.ENTER)


In [75]:

password = driver.find_element("css selector", "input[autocomplete='current-password']")
password.clear()
password.send_keys(twitter_password)
password.send_keys(Keys.ENTER)




In [76]:
tweet_test = 'https://x.com/anabelhoficial/status/1906888988192887030'
tweet_test = 'https://x.com/Juan_OrtizMX/status/1907289700101935494'


In [77]:
driver.get(tweet_test)


In [78]:
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    # Scroll hasta el fin 
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

    # Espera a que la página se cargue 
    time.sleep(4)


    # Recalcula la altura de la página
    new_height = driver.execute_script("return document.body.scrollHeight")

    # Si ya no aumenta la altura, se termina el bucle
    if new_height == last_height:
        break
    last_height = new_height

In [79]:
comments = driver.find_elements("css selector", "div[data-testid='tweetText']")
comments[0].text
#driver.find_element(By.ID, 'tweetText')
len(comments)

4

In [65]:
tweets = driver.find_elements("css selector", "article[data-testid='tweet']")
tweets[0].text



'Don Ricardo Salinas Pliego\n@RicardoBSalinas\n·\n32s\nMi estimada \n@avieu\n me mandó este muñequito diseñado con IA, ¿debería mandarlos hacer y venderlos? .\n\nLleve su #TioRichieDeLaSuerte\n1\n2\n5\n76'

In [60]:
len(tweets)


3

In [48]:
len(tweets)

17

In [37]:
#names = driver.find_elements('css selector', "span[class='css-1jxf684 r-bcqeeo r-1ttztb7 r-qvutc0 r-poiln3']")
names = driver.find_elements("css selector", "div[data-testid='User-Name']")

#names = driver.find_elements(By.XPATH, 
#                             '//*[@id="id__8hzjlqcfwha"]/div[1]/div/a/div/div[1]/span/span')

In [38]:
for x in names:
    print(x.text)


Anabel Hernández
@anabelhoficial
xrp.gerry.xah
@gerryxrp
·
Mar 31
Grok
@grok
·
Mar 31
Rossana Villares
@RossanaVillares
·
22h
Tere
@Teehreh
·
16h
Armando Galindo Reyes
@armandgal65
·
2h
Gilberto Astorga F
@vzlaastor2025
·
13h
Rena Cornejo
@cornejo_rena
·
23h
Juan Carlos Cortés
@jccortes
·
Apr 1
Kubala
@cubalamx
·
12h
Victor manuel Vazquez guerrero
@Victorm06387347
·
12h
Astrophisic
@culiacan929
·
Mar 31
Nadia
@politicanadia
·
18h
Luz Maria Saltijeral
@Luzmasaltijeral
·
Apr 1


In [25]:
#comments = driver.find_elements("css selector", "div[id='tweetText']")


In [26]:
for comment in comments:
    print(comment.text)




Siempre estaré agradecida con cada uno de mis lectores, por aquellos que decidieron conocer la verdad de los hechos ante las situaciones que afectan al país. Gracias por apreciar mi trabajo en "La historia secreta: AMLO y el Cártel de Sinaloa".
Hey 
@grok
 los libros de 
@anabelhoficial
 son ciencia ficción?
No, los libros de Anabel Hernández no son ciencia ficción. Son obras de periodismo investigativo basadas en hechos reales, como "La Historia Secreta: AMLO y el Cártel de Sinaloa", que explora corrupción y narcotráfico en México con evidencia documental, no elementos
Anabel estás correspondida !
La periodista  Anavel si pasara a la historia  de México.
Excelente libro
#PobreBurra3D
Señora,
¿No le gustaría conocer a Jaime Maussan.
Pienso que tendrían mucho de que platicar.
                                    
Parece que se acabó el contrato con la editorial... ¿Es despedía? ¿De qué verdad habla?

Tu ciencia ficción esta muy fumada, chance deberíamos meternos lo que tu te metes para e

In [ ]:
#username = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'input[autocomplete="username"]')))
#username.send_keys("your_username")
#username.send_keys(Keys.ENTER)


In [ ]:
'''
import time
from selenium import webdriver
from selenium.webdriver import ChromeOptions, Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait


options = ChromeOptions()
options.add_argument("--start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])

driver = webdriver.Chrome(options=options)
url = "https://twitter.com/i/flow/login"
driver.get(url)

username = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'input[autocomplete="username"]')))
username.send_keys("your_username")
username.send_keys(Keys.ENTER)

password = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'input[name="password"]')))
password.send_keys("your_password")
password.send_keys(Keys.ENTER)

time.sleep(10)
'''

In [ ]:
x = 'https://x.com/i/flow/login'